## PGX - Phuture GameFi Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0.1e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 1 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["gaming"]

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                -0.12,
                                100000,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    add_category_assets=['decentraland'],
    remove_category_assets=['floki','bluzelle','xai-blockchain  ']
    )
liquidity

gala on ethereum returned an invalid API response
gala recorded no valid response across any platform
ronin not supported
harmony-shard-0 not supported
energi not supported
harmony-shard-0 not supported
energi not supported
energi not supported
harmony-shard-0 not supported
energi not supported
solana not supported
Manually calculating slippage for stepn on ethereum
ronin not supported
harmony-shard-0 not supported
vanar-chain on polygon-pos returned an invalid API response
klay-token not supported
aptos not supported
okex-chain not supported
Manually calculating slippage for alien-worlds on ethereum
wax not supported
energi not supported
near-protocol not supported
cardano not supported
Manually calculating slippage for deapcoin on avalanche
solana not supported
the-open-network not supported
punk-2 on ethereum returned an invalid API response
punk-2 recorded no valid response across any platform
solana not supported
solana not supported
Manually calculating slippage for aurory on arb

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,polygon-pos,optimal chain,best slippage,optimal slippage,best slippage chain
id,,,,,,,,,,
illuvium,NaN,NaN,NaN,NaN,-0.007986,NaN,ethereum,-0.007986,-0.007986,ethereum
echelon-prime,NaN,NaN,-0.046658,NaN,-0.011895,NaN,base,-0.011895,-0.046658,ethereum
beam-2,NaN,NaN,NaN,-0.199154,-0.017460,NaN,ethereum,-0.017460,-0.017460,ethereum
metahero,NaN,NaN,NaN,-0.018305,NaN,NaN,binance-smart-chain,-0.018305,-0.018305,binance-smart-chain
mobox,NaN,NaN,NaN,-0.020686,NaN,NaN,binance-smart-chain,-0.020686,-0.020686,binance-smart-chain
...,...,...,...,...,...,...,...,...,...,...
cocos-bcx,NaN,NaN,NaN,-0.999999,-0.999810,NaN,None,-0.999810,-0.999810,ethereum
catecoin,NaN,NaN,NaN,-1.000000,-1.000000,NaN,None,-1.000000,-1.000000,binance-smart-chain
phantasma,NaN,NaN,NaN,-1.000000,NaN,NaN,None,-1.000000,-1.000000,binance-smart-chain


## Show Results

In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
immutable-x,IMX,3342124738,0.301444,19756,0xf57e7e7c23978c3caec3c3548e3d615c346e79ff,ethereum
beam-2,BEAM,1385332765,0.253728,16629,0x62d0a8458ed7719fdaf978fe5929c6d342b0bfce,ethereum
the-sandbox,SAND,1019652885,0.186752,12239,0xbbba073c31bf03b8acf7c28ef0738decf3695683,polygon-pos
illuvium,ILV,580739899,0.106364,6971,0x767fe9edc9e0df98e07454847909b5e959d7ca0e,ethereum
magic,MAGIC,204006111,0.037364,2449,0x539bde0d7dbd336b79148aa742883198bbf60342,arbitrum-one
wilder-world,WILD,175791094,0.032197,2111,0x2a3bff78b79a009976eea096a51a948a3dc00e34,ethereum
radio-caca,RACA,94468133,0.017302,1133,0x12bb890508c125661e03b09ec06e404bc9289040,binance-smart-chain
mobox,MBOX,93888658,0.017196,1126,0x3203c9e46ca618c8c1ce5dc67e7e9d75f5da2377,binance-smart-chain
aavegotchi,GHST,81075306,0.014849,973,0x385eeac5cb85a38a9a07a70c73e0a3271cfb54a7,polygon-pos
